# Population Management

### Here we'll collect information about patients with hypertension and group them into age groups

In [1]:
!jupyter nbconvert --to script FHIR_HSPC

[NbConvertApp] Converting notebook FHIR_HSPC.ipynb to script
[NbConvertApp] Writing 3794 bytes to FHIR_HSPC.py


In [2]:
import json
import requests

### The first thing to do is to look at the data. Check out the URL below to see what we're working with
https://api3.hspconsortium.org/HSPCplusSynthea/open/Condition?code=38341003&_count=50

Pull the first 50 cases of hypertension and get the total number of cases

In [38]:
response = requests.get("https://api3.hspconsortium.org/HSPCplusSynthea/open/Condition?code=38341003&_count=50")
# The _count parameter is set to it's max of 50, meaning the url will give us 50 results at a time
json_data = json.loads(response.text)
total = json_data["total"]
total  #Shows how many instances of hypertension exist in the server
json_data

{'entry': [{'fullUrl': 'https://api3.hspconsortium.org/HSPCplusSynthea/open/Condition/245763',
   'resource': {'clinicalStatus': 'active',
    'code': {'coding': [{'code': '38341003',
       'display': 'Hypertension',
       'system': 'http://snomed.info/sct'}]},
    'context': {'reference': 'Encounter/245762'},
    'id': '245763',
    'meta': {'lastUpdated': '2017-03-27T04:58:57.000+00:00', 'versionId': '1'},
    'onsetDateTime': '1991-05-02T00:59:24-04:00',
    'resourceType': 'Condition',
    'subject': {'reference': 'Patient/245761'},
    'verificationStatus': 'confirmed'},
   'search': {'mode': 'match'}},
  {'fullUrl': 'https://api3.hspconsortium.org/HSPCplusSynthea/open/Condition/499718',
   'resource': {'clinicalStatus': 'active',
    'code': {'coding': [{'code': '38341003',
       'display': 'Hypertension',
       'system': 'http://snomed.info/sct'}]},
    'context': {'reference': 'Encounter/499716'},
    'id': '499718',
    'meta': {'lastUpdated': '2017-03-27T06:50:49.000+00:0

Want to pull the Patient package from fhirclient and initialize the lists that will be used to group the age groups

In [4]:
import fhirclient.models.patient as p
import datetime
from dateutil.relativedelta import relativedelta

male_to20 = []
male_20to40 = []
male_40to60 = []
male_60to70 = []
male_70to80 = []
male_80toBeyond = []
female_to20 = []
female_20to40 = [] 
female_40to60 = []
female_60to70 = []
female_70to80 = []
female_80toBeyond = []


In [5]:
count = 0  #Once you hit "total", the loop will stop
now = datetime.datetime.today()
while count < total:
    json_data = json.loads(response.text)
    entries = json_data["entry"]  #list of condition results
    next_url = json_data["link"][1]['url'] #link to the next 50 entries
    response = requests.get(next_url)
    for h in entries:
        count += 1
        patient_id = h['resource']['subject']['reference'].split('/')[1] #Need to only have the number, not the 'Patient/' part
        patient = p.Patient.read(patient_id, smart.server)
        DOB = patient.birthDate.isostring
        DOB2 = datetime.datetime.strptime(DOB, '%Y-%m-%d')
        rdelta = relativedelta(now, DOB2)
        age = rdelta.years
        if patient.gender == "male":
            if age < 20:
                male_to20.append(h)
            if age >= 20 and age < 40:
                male_20to40.append(h)
            if age >= 40 and age < 60:
                male_40to60.append(h)
            if age >= 60 and age < 70:
                male_60to70.append(h)
            if age >= 70 and age < 80:
                male_70to80.append(h)
            if age >= 80:
                male_80toBeyond.append(h)

        if patient.gender == "female":
            if age < 20:
                female_to20.append(h)
            if age >= 20 and age < 40:
                female_20to40.append(h)
            if age >= 40 and age < 60:
                female_40to60.append(h)
            if age >= 60 and age < 70:
                female_60to70.append(h)
            if age >= 70 and age < 80:
                female_70to80.append(h)
            if age >= 80:
                female_80toBeyond.append(h)


In [6]:
print(len(male_to20))
print(len(male_20to40))
print(len(male_40to60))
print(len(male_60to70))
print(len(male_70to80))
print(len(male_80toBeyond))

134
605
589
324
277
364


In [7]:
print(len(female_to20))
print(len(female_20to40))
print(len(female_40to60))
print(len(female_60to70))
print(len(female_70to80))
print(len(female_80toBeyond))

12
552
620
277
257
329


# Type 2 Diabetes

In [43]:
response = requests.get("https://api3.hspconsortium.org/HSPCplusSynthea/open/Condition?code=44054006&_count=50")
# The _count parameter is set to it's max of 50, meaning the url will give us 50 results at a time
json_data = json.loads(response.text)
total_all = json_data["total"]
total_all  #Shows how many instances of hypertension exist in the server

1917

In [44]:
high_a1c = []
other = []
unknown = []

In [ ]:
The code below works, it just talkes

In [45]:
count = 0
while count < total_all:
    json_data = json.loads(response.text)
    entries = json_data["entry"]  #list of condition results
    next_url = json_data["link"][1]['url'] #link to the next 50 entries
    response = requests.get(next_url)
    for h in entries:
        count += 1
        patient_id = h['resource']['subject']['reference'].split('/')[1]
        a1c_data = requests.get("https://api3.hspconsortium.org/HSPCplusSynthea/open/Observation?patient=Patient/" \
                                +str(patient_id)+"&code=4548-4")
        a1c_data = json.loads(a1c_data.text)
        total = a1c_data["total"]
        if total != 0:
            a1c = a1c_data["entry"][0]["resource"]["valueQuantity"]["value"]
            if a1c >= 8:
                high_a1c.append(patient_id)
            else:
                other.append(patient_id)
        else:
            unknown.append(patient_id)
                

KeyboardInterrupt: 

In [47]:
print(len(high_a1c))
print(len(other))
print(len(unknown))
print(total_all)

0
434
159
1917


In [56]:
patId = other[1]
patId = 'Patient/'+patId
patId

'Patient/712709'